# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

## TODO 

- CHECK CORRECTNESS!
- Use buffer for creating bosonic matrices

Then I can immediately start repeating exercises of 0+1


## Improvements

- Only make matrix of Bosons tensored with fermions (better PS representation)
- State ordering option for b0-f0-b1-f1-b2-f2 OR b0-b1-b2-f0-f1-f2

## "Parameters"

- Potential
- (NEW) Lattice Spacing & Sites
- (NEW) Boundary conditions
- Bosonic dof cutoff
- (NEW) State order - for efficiencies sake.
- Binary Encoding


## Plan 

- Exact diagonalization of Hamiltonian - spectrum vs "parameters"
- VQE in QISKIT - quantum procedure
- Machine Learning ground state - compare to old studies
- Gate count for trotter steps

In [1]:
import sympy as sp
from ham_to_matrix import *
from constants import *


In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(2*aLat) )
fermionNI=sp.expand( (xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( (-1)**n*sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*(-1)**n*Derivative(V(q_{n}), q_{n}) + (-1)**n*Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [5]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [6]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

-0.5*(-1)**n*Derivative(V(q_{n}), q_{n}) + (-1)**n*Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [7]:
ham=0

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    # m*q with m=1
    return qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))


for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs)).subs((-1)**n,1)
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs)).subs((-1)**n,1)
ham=sp.simplify(ham)
ham

(4.0 - 8*\chi^{\dagger}_{0}*\chi_{0} + 4*\chi^{\dagger}_{0}*\chi_{1} - 4*\chi^{\dagger}_{0}*\chi_{2} + 4*\chi^{\dagger}_{1}*\chi_{0} + 8*\chi^{\dagger}_{1}*\chi_{1} + 4*\chi^{\dagger}_{1}*\chi_{2} - 4*\chi^{\dagger}_{2}*\chi_{0} + 4*\chi^{\dagger}_{2}*\chi_{1} - 8*\chi^{\dagger}_{2}*\chi_{2} + 4*p_{0}**2 + 4*p_{1}**2 + 4*p_{2}**2 + 3*q_{0}*q_{1} - 5*q_{0}*q_{2} + 6*q_{0}**2 - 5*q_{1}*q_{0} + 3*q_{1}*q_{2} + 6*q_{1}**2 + 3*q_{2}*q_{0} - 5*q_{2}*q_{1} + 6*q_{2}**2)/8

In [8]:
#ham=0
#totHam=bosonNI+bosonI+fermionNI+fermionI
#for i in range(0,N):
#    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

#def potential(n):
#    return q[n]

#for n in range(0,N):
#    ham=ham.subs(V(q[n]),potential(n))
#ham=ham.subs(aLat,aVal).simplify().expand()
#ham

In [9]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

1/2 - \chi^{\dagger}_{0}*\chi_{0} + \chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{0}*\chi_{2}/2 + \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{1} + \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{0}/2 + \chi^{\dagger}_{2}*\chi_{1}/2 - \chi^{\dagger}_{2}*\chi_{2} + 3*a^{\dagger}_{0}*a^{\dagger}_{1}/16 - 5*a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 5*a^{\dagger}_{0}*a_{0}/8 + 3*a^{\dagger}_{0}*a_{1}/16 - 5*a^{\dagger}_{0}*a_{2}/16 + a^{\dagger}_{0}**2/8 - 5*a^{\dagger}_{1}*a^{\dagger}_{0}/16 + 3*a^{\dagger}_{1}*a^{\dagger}_{2}/16 - 5*a^{\dagger}_{1}*a_{0}/16 + 5*a^{\dagger}_{1}*a_{1}/8 + 3*a^{\dagger}_{1}*a_{2}/16 + a^{\dagger}_{1}**2/8 + 3*a^{\dagger}_{2}*a^{\dagger}_{0}/16 - 5*a^{\dagger}_{2}*a^{\dagger}_{1}/16 + 3*a^{\dagger}_{2}*a_{0}/16 - 5*a^{\dagger}_{2}*a_{1}/16 + 5*a^{\dagger}_{2}*a_{2}/8 + a^{\dagger}_{2}**2/8 + 5*a_{0}*a^{\dagger}_{0}/8 + 3*a_{0}*a^{\dagger}_{1}/16 - 5*a_{0}*a^{\dagger}_{2}/16 + 3*a_{0}*a_{1}/16 - 5*a_{0}*a_{2}/16 + a_{0}**2/8 - 5*a_{1}*a^{\da

In [10]:
print(len(hoHam.args))
print(hoHam.args[0])
print(len(hoHam.args[0].args))
type(hoHam.args[0])


46
1/2
0


sympy.core.numbers.Half

In [11]:
cutoff=4
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)

1/2 to matrix timer took: 27.5328 seconds
a^{\dagger}_{0}**2/8 to matrix timer took: 22.7929 seconds
a^{\dagger}_{1}**2/8 to matrix timer took: 24.4229 seconds
a^{\dagger}_{2}**2/8 to matrix timer took: 25.9828 seconds
a_{0}**2/8 to matrix timer took: 25.7276 seconds
a_{1}**2/8 to matrix timer took: 27.8256 seconds
a_{2}**2/8 to matrix timer took: 32.8889 seconds
\chi^{\dagger}_{1}*\chi_{1} to matrix timer took: 46.3045 seconds
\chi^{\dagger}_{0}*\chi_{1}/2 to matrix timer took: 40.6826 seconds
\chi^{\dagger}_{1}*\chi_{0}/2 to matrix timer took: 38.9246 seconds
\chi^{\dagger}_{1}*\chi_{2}/2 to matrix timer took: 38.7343 seconds
\chi^{\dagger}_{2}*\chi_{1}/2 to matrix timer took: 38.0766 seconds
-\chi^{\dagger}_{0}*\chi_{0} to matrix timer took: 42.0039 seconds
-\chi^{\dagger}_{2}*\chi_{2} to matrix timer took: 36.6935 seconds
-5*a^{\dagger}_{0}*a^{\dagger}_{2}/16 to matrix timer took: 36.8917 seconds
-5*a^{\dagger}_{0}*a_{2}/16 to matrix timer took: 37.2030 seconds
-5*a^{\dagger}_{1}*a

In [73]:
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

print(np.isreal(ens).all())

ens

True


array([ 0.16245385+0.j,  0.63182336+0.j,  0.88924176+0.j,  1.0958083 +0.j,
        1.0975872 +0.j,  1.3383303 +0.j,  1.3511114 +0.j,  1.5561869 +0.j,
        1.5650233 +0.j,  1.7612298 +0.j,  1.8372842 +0.j,  1.9574362 +0.j,
        1.9590553 +0.j,  2.0146573 +0.j,  2.059266  +0.j,  2.1875238 +0.j,
        2.2179966 +0.j,  2.259836  +0.j,  2.2775965 +0.j,  2.4114702 +0.j,
        2.4189622 +0.j,  2.559221  +0.j,  2.6863763 +0.j,  2.7304463 +0.j,
        2.7576501 +0.j,  2.76336   +0.j,  2.7719963 +0.j,  2.8995545 +0.j,
        2.9272988 +0.j,  2.937113  +0.j,  2.9436085 +0.j,  3.1017852 +0.j,
        3.1167312 +0.j,  3.1206114 +0.j,  3.1338205 +0.j,  3.2079763 +0.j,
        3.2435696 +0.j,  3.3994672 +0.j,  3.4274263 +0.j,  3.594559  +0.j,
        3.6052384 +0.j,  3.6150215 +0.j,  3.6208181 +0.j,  3.6285117 +0.j,
        3.7612083 +0.j,  3.7884886 +0.j,  3.7948887 +0.j,  3.91954   +0.j,
        3.9490516 +0.j,  3.9514186 +0.j,  3.9671693 +0.j,  4.1608353 +0.j,
        4.16207   +0.j,  

In [12]:
cutoff=6
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)

1/2 to matrix timer took: 338.7977 seconds
a^{\dagger}_{0}**2/8 to matrix timer took: 348.7031 seconds
a^{\dagger}_{1}**2/8 to matrix timer took: 308.6836 seconds
a^{\dagger}_{2}**2/8 to matrix timer took: 295.2452 seconds
a_{0}**2/8 to matrix timer took: 302.1430 seconds
a_{1}**2/8 to matrix timer took: 255.5619 seconds
a_{2}**2/8 to matrix timer took: 287.6791 seconds
\chi^{\dagger}_{1}*\chi_{1} to matrix timer took: 439.3830 seconds
\chi^{\dagger}_{0}*\chi_{1}/2 to matrix timer took: 402.5252 seconds
\chi^{\dagger}_{1}*\chi_{0}/2 to matrix timer took: 456.9408 seconds
\chi^{\dagger}_{1}*\chi_{2}/2 to matrix timer took: 361.2784 seconds
\chi^{\dagger}_{2}*\chi_{1}/2 to matrix timer took: 358.2938 seconds
-\chi^{\dagger}_{0}*\chi_{0} to matrix timer took: 367.3905 seconds
-\chi^{\dagger}_{2}*\chi_{2} to matrix timer took: 490.3647 seconds
-5*a^{\dagger}_{0}*a^{\dagger}_{2}/16 to matrix timer took: 502.4827 seconds
-5*a^{\dagger}_{0}*a_{2}/16 to matrix timer took: 473.0312 seconds
-5*a

In [13]:
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

print(np.isreal(ens).all())

ens

True


array([ 0.04208978+0.j,  0.63672554+0.j,  1.0403124 +0.j, ...,
       18.925266  +0.j, 18.925266  +0.j, 19.609945  +0.j], dtype=complex64)